# Capistone project - The Battle of neighborhood - week 1 and 2
## 1.Introduction - Business probem
UK is in process of exiting EU, the real state market is in shake and unpredictible. But now, UK facing another problem and thet is global pandemic situacion. 
In order to assist homebuyers in Liverpool to make a good decision to purchase a suitable real estate, we gona use our data science powers.
We gonna solve this problem by clustering Liverpool neighborhoods so we can recommend venues and the current average price of real estate where buyers can make investment.

## 2.Data
We will use data for Liverpool properties from HM Land Registry - https://landregistry.data.gov.uk/ . 
The data will be in .csv. We will use use also data from FourSquare API interface and also we will arrange data into dataframe and use it for visualizaion.
After we combine data from UK land registy and data from FourSquare we will be able to easily recommend profitable real estate for investments.

## 3.Methodology
In this section we will have 4 stages:
    1.Collect data
    2.Inspection data
    3.Explore and understand data
    4.Modeling data


### 3.1.Collect data
Frst step is to import libraries and modules.

In [4]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


 
 
 ***
 ### 3.2.Inspection data
 After we import all nessesery libraries we need to get data and we use data from this link: http://landregistry.data.gov.uk/
 Next step is to see, explore and understand data.

 ***
 

In [5]:
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2020.csv")

***
### 3.3.Explore and understand data
After we fach data we need to see and try to understand and explore data. We need to put data into panda dataframe first so we can show them. 
***

In [6]:
df_ppd.head(5) 

,{A2479555-7EE0-74C7-E053-6B04A8C0887D},126000,2020-02-14 00:00,NN4 9DE,S,N,L,6,Unnamed: 8,TILLER WAY,Unnamed: 10,NORTHAMPTON,NORTHAMPTON.1,NORTHAMPTONSHIRE,A,A.1
0,{A2479555-7EE1-74C7-E053-6B04A8C0887D},190000,2020-02-07 00:00,NN8 1RD,T,N,F,35,NaN,NEPTUNE ROAD,NaN,WELLINGBOROUGH,WELLINGBOROUGH,NORTHAMPTONSHIRE,A,A
1,{A2479555-7EE2-74C7-E053-6B04A8C0887D},124999,2020-02-21 00:00,NN3 3JB,F,N,L,"WESTONIA COURT, 582 - 592",1,WELLINGBOROUGH ROAD,NaN,NORTHAMPTON,NORTHAMPTON,NORTHAMPTONSHIRE,A,A
2,{A2479555-7EE3-74C7-E053-6B04A8C0887D},235000,2020-02-14 00:00,NN5 6FY,S,N,F,14,NaN,EINSTEIN CRESCENT,DUSTON,NORTHAMPTON,NORTHAMPTON,NORTHAMPTONSHIRE,A,A
3,{A2479555-7EE4-74C7-E053-6B04A8C0887D},165000,2020-02-12 00:00,NN3 2JT,T,N,F,6,NaN,CHESTNUT TERRACE,NaN,NORTHAMPTON,NORTHAMPTON,NORTHAMPTONSHIRE,A,A
4,{A2479555-7EE6-74C7-E053-6B04A8C0887D},232000,2020-02-28 00:00,NN5 5AZ,T,N,F,11,NaN,OLD SCHOLARS CLOSE,NaN,NORTHAMPTON,NORTHAMPTON,NORTHAMPTONSHIRE,A,A


In [7]:
df_ppd.shape

(263573, 16)

***
### 3.4Modelling data
We need to set column names, format date column, clean the data, sort data, set custommers budget's max and min limits and finaly display dataframe with data. 
***

In [10]:
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [11]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2019
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2019].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [12]:
df_ppd_liverpool = df_ppd.query("Town_City == 'LIVERPOOL'")

# Make a list of street names in LIVERPOOL
streets = df_ppd_liverpool['Street'].unique().tolist()
df_grp_price = df_ppd_liverpool.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

#Customers budget's max and min limit - Find the locations df_grp_price which fits customers budget
df_affordable = df_grp_price.query("(Avg_Price >= 300000) & (Avg_Price <= 500000)")

In [13]:
# Display the dataframe
df_affordable

,Street,Avg_Price
21,AIGBURTH ROAD,319000.000000
103,ASHTON SQUARE,435000.000000
159,BEECH PARK,341500.000000
174,BEMPTON ROAD,382250.000000
177,BENTLEY ROAD,345000.000000
...,...,...
1935,WELD BLUNDELL AVENUE,325000.000000
1963,WESTWARD VIEW,395000.000000
2025,WOODPECKER DRIVE,300000.000000
2031,WOOLTON ROAD,373064.285714


***
Aditionaly we need to import some aditional librares, then we will check data.

We well use geopy. Geopy is a Python client. With geopy it is easy to locate the coordinates of adresses, cities, countirs across the globe using third-party geocoders and other data sources.
***

In [14]:
#!pip install geopy
import geopy
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy import distance
#import k-means from clustering stage
from sklearn.cluster import KMeans

In [25]:
# for index, item in df_affordable.iterrows():
   # print(f"index: {index}")
   # print(f"item: {item}")
   # print(f"item.Street only: {item.Street}")

In [15]:
geolocator = Nominatim(user_agent="final_project")

In [16]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_affordable

,Street,Avg_Price,city_coord
21,AIGBURTH ROAD,319000.000000,"(53.3711181, -2.9327252)"
103,ASHTON SQUARE,435000.000000,"(51.14241095, -114.07102779846352)"
159,BEECH PARK,341500.000000,"(41.5044842, -82.0468121)"
174,BEMPTON ROAD,382250.000000,"(53.3720881, -2.9473861)"
177,BENTLEY ROAD,345000.000000,"(39.506879, -85.055176)"
...,...,...,...
1935,WELD BLUNDELL AVENUE,325000.000000,"(53.5298583, -2.9513731)"
1963,WESTWARD VIEW,395000.000000,"(53.3775105, -2.9610538)"
2025,WOODPECKER DRIVE,300000.000000,"(38.263481, -78.775746)"
2031,WOOLTON ROAD,373064.285714,"(53.3670203, -2.8792117)"


In [18]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
21,AIGBURTH ROAD,319000.000000,"(53.3711181, -2.9327252)",53.371118,-2.932725
103,ASHTON SQUARE,435000.000000,"(51.14241095, -114.07102779846352)",51.142411,-114.071028
159,BEECH PARK,341500.000000,"(41.5044842, -82.0468121)",41.504484,-82.046812
174,BEMPTON ROAD,382250.000000,"(53.3720881, -2.9473861)",53.372088,-2.947386
177,BENTLEY ROAD,345000.000000,"(39.506879, -85.055176)",39.506879,-85.055176
...,...,...,...,...,...
1935,WELD BLUNDELL AVENUE,325000.000000,"(53.5298583, -2.9513731)",53.529858,-2.951373
1963,WESTWARD VIEW,395000.000000,"(53.3775105, -2.9610538)",53.377510,-2.961054
2025,WOODPECKER DRIVE,300000.000000,"(38.263481, -78.775746)",38.263481,-78.775746
2031,WOOLTON ROAD,373064.285714,"(53.3670203, -2.8792117)",53.367020,-2.879212


In [20]:
df = df_affordable.drop(columns=['city_coord'])

In [21]:
df

,Street,Avg_Price,Latitude,Longitude
21,AIGBURTH ROAD,319000.000000,53.371118,-2.932725
103,ASHTON SQUARE,435000.000000,51.142411,-114.071028
159,BEECH PARK,341500.000000,41.504484,-82.046812
174,BEMPTON ROAD,382250.000000,53.372088,-2.947386
177,BENTLEY ROAD,345000.000000,39.506879,-85.055176
...,...,...,...,...
1935,WELD BLUNDELL AVENUE,325000.000000,53.529858,-2.951373
1963,WESTWARD VIEW,395000.000000,53.377510,-2.961054
2025,WOODPECKER DRIVE,300000.000000,38.263481,-78.775746
2031,WOOLTON ROAD,373064.285714,53.367020,-2.879212


***
We will tell geopy to take coordinates from Liverpul, UK. After that we will create map of Liverpool and then we will move to FourSqare and combine our data whit FourSqare data.
***

In [22]:
address = 'Liverpool, UK'
geolocator = Nominatim(user_agent="final_project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Liverpool are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Liverpool are 53.407154, -2.991665.


In [23]:
# create map of Liverpool using latitude and longitude values
map_liverpool = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_liverpool)  
    
map_liverpool

In [24]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'TWZGK2XJZKXFLJX40A2YD30KDS5RUYVXOPQQUHQJMVSCXQ2U' # Foursquare ID
CLIENT_SECRET = 'VRS4WM0DBAZHJ4Q04FSSGH0MKQKOGKZHKKOSYBZJBKLIRDTX' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

***
We will use api, add our coordinates and return only  relevant information about each nearby venue.
Then we will create couple of new dataframes. Distribute data in clusters of five. Use k-means clustering and finaly create a map
***

In [25]:
  def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):  
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

AIGBURTH ROAD
ASHTON SQUARE
BEECH PARK
BEMPTON ROAD
BENTLEY ROAD
BERWICK DRIVE
BRADFIELD ROAD
BRICKFIELDS
BROWS LANE
CALDERSTONES ROAD
CARDWELL ROAD
CHALFONT ROAD
CHAPEL LANE
CHEQUERS GARDENS
CHILDWALL PARK AVENUE
CHURCH GREEN
CROSBY ROAD SOUTH
CROWN CLOSE
DARBY ROAD
DE HAVILLAND DRIVE
DUNBABIN ROAD
EAST PRESCOT ROAD
EASTERN DRIVE
EASTWOOD
ELDER GARDENS
ELM HALL DRIVE
ENNISMORE ROAD
ESHE ROAD NORTH
ESKDALE DRIVE
FAIRHOLME ROAD
FALKNER STREET
FAWLEY ROAD
FLORAL WOOD
FOXHOUSE LANE
FRIARS WAY
GLENROSE ROAD
GRABURN ROAD
GRANTON CLOSE
GREEN LANE NORTH
GREENBANK ROAD
GREENLEIGH ROAD
GWYDRIN ROAD
HALSALL LANE
HARINGTON GREEN
HENLEY ROAD
HEYDALE ROAD
HIGH PARK STREET
HIGHVILLE ROAD
HILLSIDE DRIVE
HILLVIEW GARDENS
HOLLY BANK AVENUE
HOLLYTREE ROAD
HOLMEFIELD AVENUE
HOLMWOOD DRIVE
HORRINGFORD ROAD
HUNTERS LANE
IBBOTSONS LANE
ISLAND ROAD
ISLINGTON SQUARE
JUBILEE ROAD
KENILWORTH ROAD
KESWICK ROAD
KYLEMORE AVENUE
LILLEY ROAD
LITHERLAND PARK
LONDON ROAD
LYNMOUTH ROAD
MAUNDERS COURT
MERRION CLOSE
MERS

In [27]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AIGBURTH ROAD,53.371118,-2.932725,Steves Chippy,53.373487,-2.934005,Fast Food Restaurant
1,AIGBURTH ROAD,53.371118,-2.932725,Boots,53.373273,-2.933038,Pharmacy
2,AIGBURTH ROAD,53.371118,-2.932725,Otterspool park,53.370426,-2.934281,Park
3,ASHTON SQUARE,51.142411,-114.071028,Pho Van,51.141925,-114.070443,Vietnamese Restaurant
4,ASHTON SQUARE,51.142411,-114.071028,Jerusalem Shawarma Donair & Flafel,51.142881,-114.071224,Middle Eastern Restaurant
...,...,...,...,...,...,...,...
969,YORK CLOSE,52.422609,-1.926244,Cotteridge Park,52.421417,-1.928133,Park
970,YORK CLOSE,52.422609,-1.926244,Akrams Curry House,52.424166,-1.921579,Indian Restaurant
971,YORK CLOSE,52.422609,-1.926244,Tesco Express,52.423184,-1.932913,Grocery Store
972,YORK CLOSE,52.422609,-1.926244,Attic Brew Co.,52.426308,-1.925727,Brewery


In [28]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
AIGBURTH ROAD,3,3,3,3,3,3
ASHTON SQUARE,15,15,15,15,15,15
BEECH PARK,12,12,12,12,12,12
BEMPTON ROAD,3,3,3,3,3,3
BERWICK DRIVE,4,4,4,4,4,4
...,...,...,...,...,...,...
WAYLANDS DRIVE,5,5,5,5,5,5
WELD BLUNDELL AVENUE,2,2,2,2,2,2
WESTWARD VIEW,2,2,2,2,2,2


In [29]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 216 uniques categories.


In [30]:
location_venues.shape

(974, 7)

In [31]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,AIGBURTH ROAD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AIGBURTH ROAD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AIGBURTH ROAD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ASHTON SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,ASHTON SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
liverpool_grouped = venues_onehot.groupby('Street').mean().reset_index()
liverpool_grouped

,Street,ATM,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,AIGBURTH ROAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,ASHTON SQUARE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,...,0.0,0.0,0.0,0.0,0.133333,0.0,0.0,0.000000,0.000000,0.0
2,BEECH PARK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.083333,0.0
3,BEMPTON ROAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,BERWICK DRIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,WAYLANDS DRIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
103,WELD BLUNDELL AVENUE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
104,WESTWARD VIEW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
105,WOOLTON ROAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [33]:
liverpool_grouped.shape

(107, 217)

In [34]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in liverpool_grouped['Street']:
    print("----"+hood+"----")
    temp = liverpool_grouped[liverpool_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AIGBURTH ROAD----
                           venue  freq
0                       Pharmacy  0.33
1           Fast Food Restaurant  0.33
2                           Park  0.33
3                            ATM  0.00
4  Paper / Office Supplies Store  0.00


----ASHTON SQUARE----
                       venue  freq
0                   Pharmacy  0.13
1      Vietnamese Restaurant  0.13
2  Middle Eastern Restaurant  0.07
3                 Sports Bar  0.07
4                Supermarket  0.07


----BEECH PARK----
                venue  freq
0                 Bar  0.17
1  Italian Restaurant  0.17
2         Gas Station  0.08
3          Sports Bar  0.08
4  Chinese Restaurant  0.08


----BEMPTON ROAD----
                           venue  freq
0                           Park  0.33
1               Business Service  0.33
2                          Trail  0.33
3  Paper / Office Supplies Store  0.00
4                       Mountain  0.00


----BERWICK DRIVE----
           venue  freq
0        Dog Run 

In [35]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [37]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = liverpool_grouped['Street']

for ind in np.arange(liverpool_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(liverpool_grouped.iloc[ind, :], num_top_venues)

In [38]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AIGBURTH ROAD,Pharmacy,Park,Fast Food Restaurant,Yoga Studio,Film Studio,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food
1,ASHTON SQUARE,Pharmacy,Vietnamese Restaurant,Hong Kong Restaurant,Sports Bar,Middle Eastern Restaurant,Supermarket,Convenience Store,Sandwich Place,Asian Restaurant,BBQ Joint
2,BEECH PARK,Italian Restaurant,Bar,Chinese Restaurant,Diner,Sports Bar,Gas Station,Liquor Store,Business Service,Wings Joint,Pizza Place
3,BEMPTON ROAD,Business Service,Trail,Park,Yoga Studio,Film Studio,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food
4,BERWICK DRIVE,Dog Run,Cafeteria,Historic Site,Coffee Shop,Yoga Studio,Fish & Chips Shop,Food Truck,Food Stand,Food Court,Food & Drink Shop


In [39]:
venues_sorted.shape

(107, 11)

In [40]:
liverpool_grouped.shape

(107, 217)

In [41]:
liverpool_grouped=df

In [42]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

liverpool_grouped_clustering = liverpool_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(liverpool_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([2, 3, 0, 4, 0, 1, 4, 0, 4, 0, 0, 2, 3, 2, 3, 4, 3, 4, 4, 0, 1, 0,
       4, 0, 4, 0, 2, 4, 4, 0, 0, 1, 1, 2, 2, 3, 2, 3, 2, 3, 2, 1, 0, 2,
       0, 0, 4, 2, 2, 3], dtype=int32)

In [43]:
#Dataframe to include Clusters
liverpool_grouped_clustering=df
liverpool_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
21,AIGBURTH ROAD,319000.0,53.371118,-2.932725
103,ASHTON SQUARE,435000.0,51.142411,-114.071028
159,BEECH PARK,341500.0,41.504484,-82.046812
174,BEMPTON ROAD,382250.0,53.372088,-2.947386
177,BENTLEY ROAD,345000.0,39.506879,-85.055176


In [44]:
liverpool_grouped_clustering.shape

(126, 4)

In [45]:
df.shape

(126, 4)

In [46]:
liverpool_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [47]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [48]:
# add clustering labels
liverpool_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge liverpool_grouped with liverpool_data to add latitude/longitude for each neighborhood
liverpool_grouped_clustering = liverpool_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

liverpool_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,AIGBURTH ROAD,319000.000000,53.371118,-2.932725,2,Pharmacy,Park,Fast Food Restaurant,Yoga Studio,Film Studio,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food
103,ASHTON SQUARE,435000.000000,51.142411,-114.071028,3,Pharmacy,Vietnamese Restaurant,Hong Kong Restaurant,Sports Bar,Middle Eastern Restaurant,Supermarket,Convenience Store,Sandwich Place,Asian Restaurant,BBQ Joint
159,BEECH PARK,341500.000000,41.504484,-82.046812,0,Italian Restaurant,Bar,Chinese Restaurant,Diner,Sports Bar,Gas Station,Liquor Store,Business Service,Wings Joint,Pizza Place
174,BEMPTON ROAD,382250.000000,53.372088,-2.947386,4,Business Service,Trail,Park,Yoga Studio,Film Studio,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food
177,BENTLEY ROAD,345000.000000,39.506879,-85.055176,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,BERWICK DRIVE,475000.000000,41.547161,-93.580655,1,Dog Run,Cafeteria,Historic Site,Coffee Shop,Yoga Studio,Fish & Chips Shop,Food Truck,Food Stand,Food Court,Food & Drink Shop
239,BRADFIELD ROAD,395000.000000,53.403897,-1.498421,4,Discount Store,Pub,Tram Station,Sandwich Place,Hotel,Supermarket,Park,Racetrack,Coffee Shop,Restaurant
251,BRICKFIELDS,342000.000000,3.128857,101.684553,0,Indian Restaurant,Hotel,Coffee Shop,Convenience Store,Malay Restaurant,Chinese Restaurant,Food Court,Café,Gym,South Indian Restaurant
275,BROWS LANE,400000.000000,46.262860,-63.114961,4,Sandwich Place,Pharmacy,Coffee Shop,Restaurant,Golf Course,Yoga Studio,Farm,Food & Drink Shop,Food,Flower Shop
297,CALDERSTONES ROAD,360000.000000,53.384494,-2.897299,0,Garden,Ice Cream Shop,Café,Park,Yoga Studio,Film Studio,Food Stand,Food Court,Food & Drink Shop,Food


In [49]:
# Create Map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(liverpool_grouped_clustering['Latitude'], liverpool_grouped_clustering['Longitude'], liverpool_grouped_clustering['Street'], liverpool_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [84]:
liverpool_grouped_clustering.loc[liverpool_grouped_clustering['Cluster Labels'] == 0, liverpool_grouped_clustering.columns[[1] + list(range(5, liverpool_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
159,341500.0,Bar,Italian Restaurant,Diner,Business Service,Liquor Store,Sports Bar,Gas Station,Chinese Restaurant,Pizza Place,Wings Joint
177,345000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,342000.0,Indian Restaurant,Convenience Store,Hotel,Coffee Shop,Chinese Restaurant,Food Court,South Indian Restaurant,Café,Malay Restaurant,Flower Shop
297,360000.0,Garden,Park,Ice Cream Shop,Café,Yoga Studio,English Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Film Studio
316,355000.0,Platform,Pool,Coffee Shop,Café,Dosa Place,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farm


In [85]:
liverpool_grouped_clustering.loc[liverpool_grouped_clustering['Cluster Labels'] == 1, liverpool_grouped_clustering.columns[[1] + list(range(5, liverpool_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
182,475000.0,Historic Site,Coffee Shop,Cafeteria,Dog Run,Yoga Studio,English Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
568,470000.0,Pub,Café,College Cafeteria,Indian Restaurant,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio
668,467500.0,Pub,Yoga Studio,Dosa Place,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farm,Exhibit
697,487250.0,Restaurant,Lake,Scenic Lookout,Yoga Studio,Dosa Place,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farm
823,470000.0,Café,College Cafeteria,Pub,Ice Cream Shop,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio


In [86]:
liverpool_grouped_clustering.loc[liverpool_grouped_clustering['Cluster Labels'] == 2, liverpool_grouped_clustering.columns[[1] + list(range(5, liverpool_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,319000.0,Pharmacy,Bus Stop,Park,Fast Food Restaurant,Bakery,Yoga Studio,English Restaurant,Flea Market,Fish Market,Fish & Chips Shop
341,329000.0,American Restaurant,Café,Gastropub,Grocery Store,Yoga Studio,English Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
363,300000.0,Cricket Ground,Italian Restaurant,Wine Bar,Coffee Shop,Train Station,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant
638,307350.0,Fast Food Restaurant,Pharmacy,Discount Store,Shopping Plaza,Mexican Restaurant,Supermarket,Grocery Store,Auto Garage,Coffee Shop,Diner
710,300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
liverpool_grouped_clustering.loc[liverpool_grouped_clustering['Cluster Labels'] == 3, liverpool_grouped_clustering.columns[[1] + list(range(5, liverpool_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
103,435000.000000,Pharmacy,Vietnamese Restaurant,Indian Restaurant,Convenience Store,Coffee Shop,Sandwich Place,Middle Eastern Restaurant,Sports Bar,Supermarket,Gas Station
348,452000.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,452500.000000,Convenience Store,Hotel,Steakhouse,Fast Food Restaurant,Chinese Restaurant,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
475,427833.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,450000.000000,Pub,Park,Garden Center,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant


In [88]:
liverpool_grouped_clustering.loc[liverpool_grouped_clustering['Cluster Labels'] == 4, liverpool_grouped_clustering.columns[[1] + list(range(5, liverpool_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
174,382250.0,Park,Trail,Yoga Studio,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant
239,395000.0,Pub,Tram Station,Sandwich Place,Pizza Place,Restaurant,Supermarket,Fast Food Restaurant,Shopping Mall,Bus Station,Discount Store
275,400000.0,Sandwich Place,Golf Course,Pharmacy,Coffee Shop,Restaurant,Donut Shop,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farm
388,380000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,380000.0,History Museum,Pub,Historic Site,Hotel,Café,Yoga Studio,Dosa Place,Fish Market,Fish & Chips Shop,Film Studio
